# Helper functions

Function to request frames, using the HW trigger.

TrigPeriod = in steps of 10ns.
Example: 50000 = 2kfps

In [4]:
def getFrames(DATADEBUG, numFrames = 2048, TrigPeriod = 50000):

    # damage control
    DATADEBUG.enableDataDebug(True)
    DATADEBUG.cleanData()
    TRIG.StopTriggers()
    
    #sets TriggerRegisters
    TRIG.AcqCountReset()
    TRIG.numberTrigger.set(numFrames)
    TRIG.AutoTrigPeriod.set(TrigPeriod) #in steps of 10ns
    TRIG.AutoRunEn.set(True)
    TRIG.RunTriggerEnable.set(True)
    TRIG.AutoDaqEn.set(True)
    TRIG.DaqTriggerEnable.set(True)

    while (TRIG.AcqCount.get() != numFrames) :
        print("Triggers sent: {}".format(TRIG.AcqCount.get()) , end='\r')
        time.sleep(0.1)

    # stops triggers
    TRIG.StopTriggers()
    time.sleep(0.1)

    while ( numFrames > DATADEBUG.getData().shape[2]) :
        time.sleep(1)
        print("Descrambled {}".format(DATADEBUG.getData().shape[2]), end='\r')
    print("Data Descrambled")
    print(DATADEBUG.getData().shape)

    # Print the status
    DeltaAcqCount = TRIG.AcqCount.get()
    DeltaDaqCount = TRIG.DaqCount.get()
    
    print( f'Delta Aquisition Counter: {DeltaDaqCount}')
    print( f'Number of frames received: {DATADEBUG.getData().shape}' )

    return DATADEBUG.getData()


In [1]:
def noise_print(offset = 0, imgDesc=[]):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,:,offset:offset+192].astype('float64')
    hg_n -= np.median(hg_n, axis=0, keepdims=True)
    hg_n -= np.median(hg_n, axis=(1,2), keepdims=True)
    hg_n -= np.median(hg_n, axis=2, keepdims=True) # rows (cols)
    hg_n -= np.median(hg_n, axis=1, keepdims=True) # cols (rows)
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDesc,0)
    
    #noiseMap = np.std(imgDesc[:,:,offset:offset+192],0)
    #darkAvg  = np.mean(imgDesc[:,:,192:384],0)
    medianNoise = np.median(noiseMap)
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(noiseMap, cmap='gray', vmin=0, vmax=50)
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = noiseMap
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    #plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    print()
    return noiseMap, darkAvg

def noise_print_roi(x=0,y=0,width=144,height=192,imgDesc=[]):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,y:y+height,x:x+width].astype('float64')
    hg_n -= np.median(hg_n, axis=0, keepdims=True)
    hg_n -= np.median(hg_n, axis=(1,2), keepdims=True)
    hg_n -= np.median(hg_n, axis=2, keepdims=True) # rows (cols)
    hg_n -= np.median(hg_n, axis=1, keepdims=True) # cols (rows)
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDesc,0)
    
    #noiseMap = np.std(imgDesc[:,:,offset:offset+192],0)
    #darkAvg  = np.mean(imgDesc[:,:,192:384],0)
    medianNoise = np.median(noiseMap)
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(noiseMap, cmap='gray', vmin=-50, vmax=50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = noiseMap
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    print()
    return noiseMap, darkAvg

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [1]:
def glitchFinder_print(x=0,y=0,width=144,height=192):
    print(imgDesc[100].shape)
    hg_n  = imgDesc[:,y:y+height,x:x+width].astype('float64')
    imgDescCMC = hg_n
    noiseMap = np.std(imgDescCMC,0)
    darkAvg  = np.mean(imgDescCMC,0)
    
    medianNoise = np.median(noiseMap)
    refDarkValue = int(np.mean(np.mean(darkAvg)))
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDescCMC[0], cmap='gray', vmin=refDarkValue-50, vmax=refDarkValue+50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)

    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = imgDescCMC
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    plt.figure(1,figsize=(6,4),dpi=150)
    plt.imshow(imgDescCMC[0]-darkAvg, cmap='gray', vmin=-50, vmax=50, interpolation='none')
    plt.colorbar(location='right', anchor=(0, 0.3), shrink=0.7)
    
    plt.figure(2,figsize=(6,4),dpi=150)
    dataset = imgDescCMC - darkAvg 
    plt.hist(dataset.reshape(-1), bins=100, label=f'\nNoise Map distribution AHL, Avg. Dark level w/o CMC %d, Mediam Noise %d' % (np.mean(np.mean(darkAvg)), medianNoise)) 
    plt.yscale('log')
    plt.title("histogram") 
    plt.legend()
    plt.show()
    
    
    return noiseMap, darkAvg

# Hack to get lanes to behave
Request data and disable lanes which fail

In [1]:
def lanesHackTest(root, PKREG, DATADEBUG):
    PKREG.CountReset()
    for i in range(24):
        print("PKREG.TimeoutCntLane[{}]={}".format(i, PKREG.TimeoutCntLane[i].get()))
    
    # Take data
    PKREG.CountReset()
    imgDesc = run_asic_test(DATADEBUG, numFrames = 2048, initialDelay = 0)

In [3]:
def lanesHackDisable(root, PKREG, DATADEBUG):   
    TimeoutCntLane = [0] * 24
    disable = PKREG.DisableLane.get()
    for i in range(24):
        print("PKREG.TimeoutCntLane[{}]={}".format(i, PKREG.TimeoutCntLane[i].get()))
        TimeoutCntLane[i] = PKREG.TimeoutCntLane[i].get()
        if(TimeoutCntLane[i]> 20) :
            #disable lane
            disable = disable | 0x1<<i

    PKREG.DisableLane.set(disable)

    #get locked lanes
    root.getLaneLocks()

    #Disabled lanes in ASICS
    for asicIndex in range(4) : 
        root.App.AsicTop.DigAsicStrmRegisters0.enable.set(True)
        print("ASIC{}: {:#x}".format(asicIndex, getattr(root.App.AsicTop, f"DigAsicStrmRegisters{asicIndex}").DisableLane.get()))    